# Exploração dos Dados

Link: https://github.com/Ignorancia-Zero/curso-ciencia-dados-treino/blob/main/notebooks/Teste%20Unit%C3%A1rio%20-%20Amostras.ipynb

In [1]:
import gc
import os
import pyunpack
import rarfile
import shutil
import zipfile
import numpy as np
import pandas as pd
import patoolib

from pathlib import Path
from tqdm import tqdm

In [2]:
try:
    PASTA_NOTEBOOK / ""
except:
    PASTA_NOTEBOOK = Path(os.getcwd())

PASTA_PROJETO = PASTA_NOTEBOOK.parent
PASTA_DADOS = PASTA_PROJETO / "dados"
PASTA_EXTERNO = PASTA_DADOS / "externo"
PASTA_TESTE = PASTA_PROJETO / "testes"
PASTA_TESTE_CODIGO = PASTA_TESTE / "codigos"

# PASTA_TESTE_CODIGO.mkdir(exist_ok=True)
print(PASTA_DADOS)
os.chdir(PASTA_PROJETO)

c:\Users\Thiago\Desktop\Pipeline ETL\src\dados


In [ ]:
# inicializa os dados de entrada como um dicionário vazio
dados = dict()

# para cada arquivo do censo demográfico
for censo in tqdm(os.listdir(PASTA_DADOS / "zip")):

    # abre o arquivo zip com o conteúdo do censo
    with zipfile.ZipFile(PASTA_DADOS / f"zip/{censo}") as z:

        # lista os conteúdos dos arquivos zip que contém o nome tabela
        arqs = [f for f in z.namelist() if f"escolas." in f.lower()]

        # # se houver algum arquivo deste tipo dentro do zip
        if len(arqs) == 1:
            arq = arqs[0]

            # este arquivo for um CSV
            if ".csv" in arq.lower():
                # le os conteúdos do arquivo por meio do buffer do zip
                dados[censo] = pd.read_csv(
                    z.open(arq), encoding="latin-1", sep="|"
                )


            # caso seja outro arquivo zip
            elif ".zip" in arq.lower():
                # cria um novo zipfile e le o arquivo deste novo zip
                with zipfile.ZipFile(z.open(arq)) as z2:
                    arq = z2.namelist()[0]
                    dados[censo] = pd.read_csv(
                        z2.open(arq), encoding="latin-1", sep="|"
                    )


            # caso seja um arquivo winrar
            elif ".rar" in arq.lower():
                # Nome do arquivo .rar
                name_rar = censo.split('.')[-2]

                # Descompactando o arquivo
                arquivo_rar = PASTA_DADOS / f"zip/{censo}"
                diretorio_destino = PASTA_DADOS / f"zip/csv"
                patoolib.extract_archive(arquivo_rar, outdir=diretorio_destino)

                arquivo_rar = PASTA_DADOS / f"zip/csv/{name_rar}/DADOS/ESCOLAS.rar"
                diretorio_destino = PASTA_DADOS / f"zip/csv/desc/{name_rar}"
                patoolib.extract_archive(arquivo_rar, outdir=diretorio_destino)

                # Lista e filtra os arquivos CSV extraídos
                csv = [
                    f
                    for f in os.listdir(PASTA_DADOS / f"zip/csv/desc/{name_rar}")
                    if f"escolas." in f.lower()
                ][0]
                dados[censo] = pd.read_csv(
                    PASTA_DADOS / f"zip/csv/desc/{name_rar}" / f"{csv}", encoding="latin-1", sep="|"
                )
# Apaga a pasta CSV criada para descompactar os arquivos
shutil.rmtree(PASTA_DADOS / f"zip/csv")

In [15]:
# renomeia as colunas
dados[censo].rename(
    columns={
        "PK_COD_ENTIDADE": "CO_ENTIDADE",
        "PK_COD_TURMA": "ID_TURMA",
        "FK_COD_DOCENTE": "ID_DOCENTE",
        "CO_PESSOA_FISICA": "ID_DOCENTE",
        "PK_COD_MATRICULA": "ID_MATRICULA",
        "FK_COD_ALUNO": "ID_ALUNO",
        "ANO_CENSO": "NU_ANO_CENSO",
        "FK_COD_MUNICIPIO": "CO_MUNICIPIO"
    }, 
    inplace=True
)
                
# aplica os filtros sobre o arquivo
# if filtros is not None:
#     for c, l in filtros.items():
#         dados[censo] = dados[censo].loc[lambda f: f[c].isin(l)]

# seleciona as colunas
# dados[censo] = dados[censo].reindex(columns=cols).drop_duplicates()


In [16]:
df_dados.head(2)

,ANO_CENSO,PK_COD_ENTIDADE,NO_ENTIDADE,COD_ORGAO_REGIONAL_INEP,DESC_SITUACAO_FUNCIONAMENTO,FK_COD_ESTADO,SIGLA,FK_COD_MUNICIPIO,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,...,IN_ORGAO_ASS_PAIS_MESTRES,IN_ORGAO_CONSELHO_ESCOLAR,IN_ORGAO_GREMIO_ESTUDANTIL,IN_ORGAO_OUTROS,IN_ORGAO_NENHUM,TP_PROPOSTA_PEDAGOGICA,IN_EQUIP_NENHUM,QT_PROF_GESTAO,QT_PROF_ASSIST_SOCIAL,IN_MATERIAL_PED_NENHUM
0,2007.0,11017856.0,CEEJA MARECHAL RONDON,NaN,EM ATIVIDADE,11.0,RO,1100254.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007.0,11002719.0,EMEIEF PEQUENO MESTRE,NaN,EM ATIVIDADE,11.0,RO,1100205.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_dados = pd.concat(dados.values())

In [10]:
df_dados.shape

(30556, 486)

In [11]:
df_dados.isna().sum()

ANO_CENSO                      13241
PK_COD_ENTIDADE                13241
NO_ENTIDADE                        0
COD_ORGAO_REGIONAL_INEP        14732
DESC_SITUACAO_FUNCIONAMENTO    13241
                               ...  
TP_PROPOSTA_PEDAGOGICA         27494
IN_EQUIP_NENHUM                29028
QT_PROF_GESTAO                 29028
QT_PROF_ASSIST_SOCIAL          29028
IN_MATERIAL_PED_NENHUM         29028
Length: 486, dtype: int64